In [1]:
from utils import *

Let's say you've got weights that you need to use but they're not in tensorflow format (e.g. maybe from Theano or Caffe or something). If you want to put these into your tensorflow model, you'll have to first get the weights into Python, probably as a numpy array. First, I'm going to load an old model, then we'll pretend we've gotten weights from somewhere else and load them in.

In [2]:
reset_graph()

In [3]:
# loads the graph
saver = tf.train.import_meta_graph("savedmodels/11_07_gradientclipping.ckpt.meta")

Let's say we want to replace the weights in hidden layer 2 with something else. In tensorflow this is named `hidden2/kernel:0`. You can see the list of trainable variables by printing out items in `tf.trainable_variables()`. 

In [4]:
with tf.Session() as sess:
    saver.restore(sess, "savedmodels/11_07_gradientclipping.ckpt")
    q = sess.run("hidden2/kernel:0")
    print(q[:5, :5])
    print(q.shape)

INFO:tensorflow:Restoring parameters from savedmodels/11_07_gradientclipping.ckpt
[[ 0.0381882  -0.04454234 -0.0126121   0.07089889 -0.03703738]
 [ 0.11136749 -0.01331152  0.09881812 -0.01967457 -0.05764915]
 [ 0.08301214 -0.15068418  0.01724102  0.10922857 -0.06793457]
 [-0.1117208   0.03834829  0.0244196   0.01757379  0.13873045]
 [-0.04542917 -0.01718456 -0.03057014  0.10342998  0.12199068]]
(300, 50)


The weights have a shape of $300 \times 50$, representing weighted connections from the 300 neurons in hidden 1 to the 50 neurons of hidden 2. I'm just going to make up a random $300 \times 50$ matrix, but assume we've pulled it from some other model. 

In [5]:
new_weights = np.random.randn(300, 50)
print(new_weights[:5, :5])

[[ 0.49671415 -0.1382643   0.64768854  1.52302986 -0.23415337]
 [ 0.32408397 -0.38508228 -0.676922    0.61167629  1.03099952]
 [-1.41537074 -0.42064532 -0.34271452 -0.80227727 -0.16128571]
 [ 0.25049285  0.34644821 -0.68002472  0.2322537   0.29307247]
 [ 0.35778736  0.56078453  1.08305124  1.05380205 -1.37766937]]


Here's how we feed these weights into the model.

In [6]:
# Get the assign ops for hidden2
assign_kernel = tf.get_default_graph().get_operation_by_name("hidden2/kernel/Assign")
# Get the initializing assignment, which is the second item, index 1
init_kernel = assign_kernel.inputs[1]

# make a new init operation
init = tf.global_variables_initializer()

with tf.Session() as sess:
    # restore parameters
    saver.restore(sess, "savedmodels/11_07_gradientclipping.ckpt")
    print("Old hidden 2 weights:")
    print(sess.run("hidden2/kernel:0")[:5, :5])
    print("Old hidden 1 weights:")
    print(sess.run("hidden1/kernel:0")[:5, :5])
    
    # load the new weights
    sess.run(init, feed_dict={init_kernel: new_weights})
    print("New hidden 2 weights:")
    print(sess.run("hidden2/kernel:0")[:5, :5])
    print("New hidden 1 weights:")
    print(sess.run("hidden1/kernel:0")[:5, :5]) 

INFO:tensorflow:Restoring parameters from savedmodels/11_07_gradientclipping.ckpt
Old hidden 2 weights:
[[ 0.0381882  -0.04454234 -0.0126121   0.07089889 -0.03703738]
 [ 0.11136749 -0.01331152  0.09881812 -0.01967457 -0.05764915]
 [ 0.08301214 -0.15068418  0.01724102  0.10922857 -0.06793457]
 [-0.1117208   0.03834829  0.0244196   0.01757379  0.13873045]
 [-0.04542917 -0.01718456 -0.03057014  0.10342998  0.12199068]]
Old hidden 1 weights:
[[-0.07049792 -0.00975064 -0.03468948  0.03130994  0.05715947]
 [ 0.04928269 -0.05647784  0.03744679 -0.01494795 -0.01138932]
 [ 0.07077828 -0.00395048  0.01716127 -0.04993568 -0.01295266]
 [ 0.05774249 -0.00075845 -0.03363854 -0.02605219 -0.02206105]
 [ 0.02206128  0.00489597 -0.06169014  0.00238407  0.02339816]]
New hidden 2 weights:
[[ 0.49671414 -0.1382643   0.64768857  1.5230298  -0.23415338]
 [ 0.32408398 -0.38508227 -0.67692202  0.61167628  1.03099954]
 [-1.4153707  -0.42064533 -0.34271452 -0.80227727 -0.16128571]
 [ 0.25049284  0.34644821 -0.68

Note that the hidden 1 weights remain unchaged, but the hidden 2 weights have been updated with the values in `new_weights`. 